# dbt & AWS Glue demo

## Interactive Query

In [1]:
!jupyter kernelspec list

Available kernels:
  pythonjvsc74a57bd0aee8b7b246df8f9039afb4144a1f6fd8d2ca17a180786b69acc140d282b71a49    /Users/mehdmldj/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/temp/jupyter/kernels/pythonjvsc74a57bd0aee8b7b246df8f9039afb4144a1f6fd8d2ca17a180786b69acc140d282b71a49
  pythonjvsc74a57bd0ecfc9e0234ba45f0b6ad9c1de906e5da54181acbc252a1bd8855893fb3e13c93    /Users/mehdmldj/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/temp/jupyter/kernels/pythonjvsc74a57bd0ecfc9e0234ba45f0b6ad9c1de906e5da54181acbc252a1bd8855893fb3e13c93
  python3                                                                               /Users/mehdmldj/dbt_demo_bkp/venv/share/jupyter/kernels/python3
  glue_pyspark                                                                          /usr/local/share/jupyter/kernels/glue_pyspark
  glue_spark                                                                            /usr/local/share/jupyter/kernels/glue_spark


In [1]:
%glue_version 3.0
%iam_role "arn:aws:iam::788996449432:role/AWSGlueServiceRole-dbt"
%idle_timeout 120
%profile default

UsageError: Line magic function `%glue_version` not found.


In [7]:
%%configure
{"--conf":"spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog --conf spark.serializer=org.apache.spark.serializer.KryoSerializer"
,"--connection-names":"test_benjamin_delta,test_hudi_connector"
,"--extra-jars":"/tmp/*.jar"
,"--extra-py-files":"/tmp/delta-core_2.12-1.0.0.jar"
}

The following configurations have been updated: {'--conf': 'spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog --conf spark.serializer=org.apache.spark.serializer.KryoSerializer', '--connection-names': 'test_benjamin_delta,test_hudi_connector', '--extra-jars': '/tmp/*.jar', '--extra-py-files': '/tmp/delta-core_2.12-1.0.0.jar'}


In [ ]:
import sys
import boto3
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue import DynamicFrame

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::913308485840:role/GlueInteractiveSessionRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: benjamin-e42896d1-c952-462a-91b3-333d535d731f
Applying the following default arguments:
--glue_kernel_version 0.35
--enable-glue-datacatalog true
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog --conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--connection-names test_benjamin_delta,test_hudi_connector
--extra-jars /tmp/*.jar
--extra-py-files /tmp/delta-core_2.12-1.0.0.jar
Waiting for session benjamin-e42896d1-c952-462a-91b3-333d535d731f to get into ready status...
Session benjamin-e42896d1-c952-462a-91b3-333d535d731f has been created




In [2]:
%%sql
select customer_id \
,first_name \
,last_name \
,first_order \
,most_recent_order \
,number_of_orders \
,customer_lifetime_value \
from dbt_demo_01.customers \
where customer_id = '1.0'

+-----------+----------+---------+-----------+-----------------+----------------+-----------------------+
|customer_id|first_name|last_name|first_order|most_recent_order|number_of_orders|customer_lifetime_value|
+-----------+----------+---------+-----------+-----------------+----------------+-----------------------+
|        1.0|   Michael|       P.| 2018-01-01|       2022-10-05|               3|                   76.0|
+-----------+----------+---------+-----------+-----------------+----------------+-----------------------+


In [3]:
%%sql
select customer_id
,first_name
,last_name
,first_order
,most_recent_order
,number_of_orders
,dbt_valid_from
,dbt_valid_to
from dbt_demo_01.demosnapshot
where customer_id = '1.0'

+-----------+----------+---------+-----------+-----------------+----------------+--------------+------------+
|customer_id|first_name|last_name|first_order|most_recent_order|number_of_orders|dbt_valid_from|dbt_valid_to|
+-----------+----------+---------+-----------+-----------------+----------------+--------------+------------+
|        1.0|   Michael|       P.| 2018-01-01|       2018-02-10|               2|    2022-10-07|        null|
+-----------+----------+---------+-----------+-----------------+----------------+--------------+------------+
